In [246]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, confusion_matrix, accuracy_score, recall_score
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder
import datetime
import os
import json

In [247]:
# Parámetros
inputFolder = "1-input"
processFolder = "2-process"
outputFolder = "3-output"
logsFolder = "4-logs"

dataVisualizationTopLimit = 20

testSize = 0.25
randomState = 0
partitionsNumber = 0

In [248]:
# Función para leer datos de un archivo csv
def readData(filePath, delimiter, encoding, header):
  data = pd.read_csv(filePath, delimiter=delimiter,encoding=encoding, header=header)
  return data

In [249]:
def preprocessingData(df):
  # Aca no se realizará ni parseo ni nada, ya vendrá desde la fuente
  # Solo se leerá, se dividirá y se entrenará

  # Balanceo de datos: Sobremuestreo aleatorio (oversampling)
  objectiveColumn = "hired"
  class0, class1 = df[objectiveColumn].value_counts()
  dfClass0 = df[df[objectiveColumn] == 0]
  dfClass1 = df[df[objectiveColumn] == 1]
  dfClass1Sampled = dfClass1.sample(class0, replace=True)
  df = pd.concat([dfClass0, dfClass1Sampled],axis=0)

  # Desordenando la data para evitar sesgos(filas)
  df = df.sample(frac = 1, random_state=randomState).reset_index(drop=True)  

  # Aplicando OrdinalEncoding a las variables categóricas ordinales()
  categoricalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "hired" ] and columnType == "object" ]
  categoricalOrdinalColumns = [columnName for columnName in categoricalColumns if columnName in [ "lastEducationStatus", "lastEducationDegree" ]]

  # OrdinalEncoder para la columna lastEducationStatus
  encoder = OrdinalEncoder(categories=[[ "Abandonado", "En Curso", "Graduado" ]])
  encoder.fit(df[["lastEducationStatus"]])
  df["lastEducationStatus"] = encoder.transform(df[["lastEducationStatus"]])

  # OrdinalEncoder para la columna lastEducationDegree
  encoder = OrdinalEncoder(categories=[[ "Otro", "Secundario", "Terciario/Tecnico", "Universitario", "Posgrado", "Master", "Doctorado" ]])
  encoder.fit(df[["lastEducationDegree"]])
  df["lastEducationDegree"] = encoder.transform(df[["lastEducationDegree"]])

  #display(df)

  # Aplicando OneHotEncoding a las variables categóricas cardinales (transformación a numéricas mediante columnas)
  categoricalCardinalColumns = [columnName for columnName in categoricalColumns if columnName not in [ "lastEducationStatus", "lastEducationDegree" ]]
  for column in categoricalCardinalColumns:
    dummies = pd.get_dummies(df[[column]], prefix=column, dummy_na=True)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(columns=[column])

  #display(df.dtypes)

  # Aplicando MinMaxScaler a las variables numéricas (normalización) (esto tambien incluye a lastEducationStatus y lastEducationDegree, ya numéricas)
  # Algunas quedaran en 0.9999, esto porque no todas manejan la misma escala (sin decimales, o solo un decimal)
  numericalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "hired" ] and columnType == "float64" ]
  #print(numericalColumns)
  for column in numericalColumns:
    df[column] = df[column].fillna(0.0)

  mms = MinMaxScaler()
  df[numericalColumns] = mms.fit_transform(df[numericalColumns])

  # Leyendo el numero de atributos
  display(df)
  print(len(df))
  print(len(df.columns))
  
  # 19146 filas, 12403 columnas con el drop_first=True
  # 19146 filas, 12415 columnas sin el drop_first=True

  # Revision maximos y minimos (todos si estan entre 0 y 1)
  """df.loc['max'] = df.max()
  df.loc['min']= df.min()
  maxValue, minValue = max(list(df.iloc[len(df)-2])), min(list(df.iloc[len(df)-1]))
  print(maxValue, minValue)
  df = df.drop(['max', 'min'], axis=0)"""

  # Eliminando columnas con varianza cercana a cero (variables no afectan en el resultado del modelo)
  percentileNumber = 95
  df.loc['std'] = df.std()
  stdArray = df.iloc[len(df)-1]
  ninetyNinthPercentile = np.percentile(stdArray, percentileNumber)
  df = df.transpose()
  df = df[df["std"]>ninetyNinthPercentile]
  df = df.transpose()
  df = df.drop(['std'], axis=0)

  display(df)
  print(len(df))
  print(len(df.columns))

  # Lectura de las variables de características y objetivo
  objectiveColumn = "hired"
  X = df.drop([objectiveColumn], axis=1)
  y = df[objectiveColumn]

  return X, y

In [250]:
def splitData(X, y, partitionNumber):
  # Dividiendo los dataframes de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSize, random_state=randomState)

  if partitionNumber != 0:
    # Obteniendo el total de filas del dataframe de testeo
    X_train, X_test, y_train, y_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)
    totalRows = len(X_train)

    # Determinando el límite inferior (primera fila) de la partición
    bottomLimit = int(totalRows*testSize*(partitionNumber-1)) + 1

    # Determinando el límite superior (última fila) de la partición
    topLimit = int(totalRows*testSize*partitionNumber)

    # Determinando una lista con todas los numeros de las filas con la partición
    testList = [x for x in range(bottomLimit-1, topLimit)]

    # Determinando los dataframes de las categorías
    X_train, X_test = X_train[~X_train.index.isin(testList)], X_train[X_train.index.isin(testList)]

    # Determinando los dataframes de los objetivos
    y_train, y_test = y_train[~y_train.index.isin(testList)], y_train[y_train.index.isin(testList)]

  return X_train, X_test, y_train, y_test

In [251]:
def createClassifier(modelName, parameters):
  if modelName == "KNN":
    return KNeighborsClassifier(parameters) if parameters is not None else KNeighborsClassifier()
  elif modelName == "LR":
    return LogisticRegression(parameters) if parameters is not None else LogisticRegression()
  elif modelName == "GNB":
    return GaussianNB(parameters) if parameters is not None else GaussianNB()
  elif modelName == "DT":
    return DecisionTreeClassifier(parameters) if parameters is not None else DecisionTreeClassifier()
  elif modelName == "SVM":
    return SVC(parameters) if parameters is not None else SVC()
  elif modelName == "RF":
    return RandomForestClassifier(parameters) if parameters is not None else RandomForestClassifier()
  elif modelName == "GB":
    return GradientBoostingClassifier(parameters) if parameters is not None else GradientBoostingClassifier()
  else:
    return KNeighborsClassifier()

In [252]:
def trainModel(X_train, X_test, y_train, y_test, modelName, parameters = None):
  # Creación del clasificador KNN
  clf = createClassifier(modelName, parameters)

  # Entrenamiento del clasificador KNN
  clf.fit(X_train, y_train)

  # Calculando la predicción del modelo con la data de prueba
  y_pred = clf.predict(X_test)

  return y_test, y_pred

In [253]:
def getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, algorithm = "KNN"):
  trainRows = len(y_train)
  testRows = len(y_test)

  # Calculando la exactitud del modelo
  accuracy = "{:.2%}".format(accuracy_score(y_test, y_pred))

  # Calculando la precisión del modelo
  precision = "{:.2%}".format(precision_score(y_test, y_pred))

  # Calculando la sensibilidad del modelo
  recall = "{:.2%}".format(recall_score(y_test, y_pred))

  # Calculando el valor F del modelo (robustez)
  f1Score = "{:.2%}".format(f1_score(y_test, y_pred))

  # Calculando el tiempo de ejecución del modelo
  executionTime = str(int((endDate - startDate).total_seconds() * 1000)) + "ms"

  confussionMatrix = str(confusion_matrix(y_test, y_pred).tolist())
  
  return {
    "algoritmo": algorithm,
    "tipo": "Total de datos" if partitionNumber == 0 else "Particion " + str(partitionNumber),
    "registrosEntrenamiento": trainRows,
    "registrosPrueba": testRows,
    "tiempoEjecucion": executionTime,
    "matrizConfusion": confussionMatrix,
    "exactitud": accuracy,
    "precision": precision,
    "sensibilidad": recall,
    "valorF": f1Score,
  }


In [254]:
def main():
  # Definiendo el inicio del proceso
  startTime = datetime.datetime.now()
  print("Inicio: " + str(startTime))
  print("Se inició el procesamiento")
  print()

  # Leyendo la data
  data = readData(os.path.join(inputFolder, "result.csv"), ',', 'utf-8', 0)

  # Determinando los dataframes de las categorías (X) y el objetivo (y)
  X, y = preprocessingData(data)

  # Creando el arreglo de metricas de cada partición
  metricsList = []

  # Iterando sobre cada partición
  for partitionNumber in range(0,partitionsNumber+1):
    # Separando data para el entrenamiento y testeo
    X_train, X_test, y_train, y_test = splitData(X, y, partitionNumber)

    models = ["KNN", "LR", "GNB", "DT", "RF", "GB"]

    for model in models:
      print("Calculando para el algoritmo {}".format(model))

      # Inicio de ejecución del modelo
      startDate = datetime.datetime.now()
      print("Inicio: " + str(startDate))
      
      # Realizar entrenamiento del modelo
      y_test, y_pred = trainModel(X_train, X_test, y_train, y_test, model)

      # Fin de ejecución del modelo
      endDate = datetime.datetime.now()
      print("Fin: " + str(endDate))
      print("Tiempo: " + str(endDate-startDate))
      print()

      # Obteniendo las métricas de la partición del modelo
      metrics = getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, model)

      # Añadiendo la métrica de la partición a la lista de métricas
      metricsList.append(metrics)

  # Ordenando las métricas
  metricsList = sorted(metricsList, key=lambda x: (x["algoritmo"], x["tipo"]))

  # Mostrando las métricas
  metricsListPretty = json.dumps(metricsList, indent=4)
  print(metricsListPretty)
    
  # Escribiendo las metricas en un archivo de salida
  with open(os.path.join(outputFolder, datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S") + ".json"), "w") as f:
      json.dump(metricsList, f, indent=4, ensure_ascii=False)

  # Definiendo el fin del proceso
  endTime = datetime.datetime.now()
  print("Fin: " + str(endTime))
  print("Tiempo: " + str(endTime-startTime))
  
  # Retornando la lista de métricas
  return metricsList

In [255]:
if __name__ == "__main__":
  main()

Inicio: 2023-05-22 20:59:38.735859
Se inició el procesamiento



,salary,worksNumber,lastEducationStatus,lastEducationDegree,studiesNumber,technicalSkills,languages,otherSkills,hired,jobProfileName_Administrador De Soporte Tecnico,...,lastEducationName_Win Office Systen (Abaco),"lastEducationName_Windows Server 2012 R2 - Seguridad Tmg ,Firew",lastEducationName_Windows Server 2016,lastEducationName_Wordpress Con Elementor,"lastEducationName_Workshop ""Inception Agil""",lastEducationName_Workshop Machine Learning Con Spark,lastEducationName_Workshop Triangulo Del Conocimiento Del Pmi,lastEducationName_Xiii Taller De Preparacion Para El Examen Pmp,lastEducationName_Yellow Belt Of Six Sigma Methodology,lastEducationName_nan
0,0.479416,0.473684,1.0,0.500000,0.083333,1.000000,0.000000,1.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.376670,0.157895,1.0,0.000000,0.083333,0.382353,0.000000,0.252941,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.239674,0.263158,1.0,0.000000,0.416667,0.029412,0.000000,0.252941,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0.068429,0.000000,0.5,0.500000,0.000000,0.352941,0.000000,0.411765,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.582163,0.473684,0.5,0.833333,0.166667,0.411765,0.000000,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19141,0.136927,0.052632,1.0,0.500000,0.000000,0.029412,0.250000,0.252941,0,0,...,0,0,0,0,0,0,0,0,0,0
19142,0.513665,0.315789,0.5,0.833333,0.166667,0.000000,0.000000,0.117647,1,0,...,0,0,0,0,0,0,0,0,0,0
19143,0.136927,0.157895,1.0,0.500000,0.083333,0.117647,0.833333,0.117647,1,0,...,0,0,0,0,0,0,0,0,0,0
19144,0.034180,0.268421,0.5,0.500000,0.000000,0.244118,0.250000,0.252941,1,0,...,0,0,0,0,0,0,0,0,0,0


19146
12415


,salary,worksNumber,lastEducationStatus,lastEducationDegree,studiesNumber,technicalSkills,languages,otherSkills,hired,jobProfileName_Administrador De Soporte Tecnico,...,lastEducationName_Scrum Master,lastEducationName_Tecnico,lastEducationName_Tecnico En Computacion E Informatica,lastEducationName_Titulado En Ing. Informatica,lastEducationName_Titulado En Ing.Sistemas Y Computo,lastEducationName_Titulado En Ingenieria De Software,lastEducationName_Trabajo Social,lastEducationName_Transformacion Digital,lastEducationName_Universitario,lastEducationName_Usuario Experto Sap Mm
0,0.479416,0.473684,1.0,0.500000,0.083333,1.000000,0.000000,1.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.376670,0.157895,1.0,0.000000,0.083333,0.382353,0.000000,0.252941,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.239674,0.263158,1.0,0.000000,0.416667,0.029412,0.000000,0.252941,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.068429,0.000000,0.5,0.500000,0.000000,0.352941,0.000000,0.411765,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.582163,0.473684,0.5,0.833333,0.166667,0.411765,0.000000,0.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19141,0.136927,0.052632,1.0,0.500000,0.000000,0.029412,0.250000,0.252941,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19142,0.513665,0.315789,0.5,0.833333,0.166667,0.000000,0.000000,0.117647,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19143,0.136927,0.157895,1.0,0.500000,0.083333,0.117647,0.833333,0.117647,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19144,0.034180,0.268421,0.5,0.500000,0.000000,0.244118,0.250000,0.252941,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


19146
617
Calculando para el algoritmo KNN
Inicio: 2023-05-22 20:59:43.520637
Fin: 2023-05-22 20:59:44.024410
Tiempo: 0:00:00.503773

Calculando para el algoritmo LR
Inicio: 2023-05-22 20:59:44.037366


c:\Users\Ronaldo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fin: 2023-05-22 20:59:44.733911
Tiempo: 0:00:00.696545

Calculando para el algoritmo GNB
Inicio: 2023-05-22 20:59:44.742875
Fin: 2023-05-22 20:59:44.915906
Tiempo: 0:00:00.173031

Calculando para el algoritmo DT
Inicio: 2023-05-22 20:59:44.924874
Fin: 2023-05-22 20:59:45.507683
Tiempo: 0:00:00.582809

Calculando para el algoritmo RF
Inicio: 2023-05-22 20:59:45.516639
Fin: 2023-05-22 20:59:49.314113
Tiempo: 0:00:03.797474

Calculando para el algoritmo GB
Inicio: 2023-05-22 20:59:49.323083
Fin: 2023-05-22 20:59:56.835389
Tiempo: 0:00:07.512306

[
    {
        "algoritmo": "DT",
        "tipo": "Total de datos",
        "registrosEntrenamiento": 14359,
        "registrosPrueba": 4787,
        "tiempoEjecucion": "582ms",
        "matrizConfusion": "[[2227, 176], [0, 2384]]",
        "exactitud": "96.32%",
        "precision": "93.12%",
        "sensibilidad": "100.00%",
        "valorF": "96.44%"
    },
    {
        "algoritmo": "GB",
        "tipo": "Total de datos",
        "registrosE